In [3]:
import pandas as pd
from datetime import datetime, timedelta
from data_mountain_query.counters import AmbientTweetCounters, Counters
from data_mountain_query.sentiment import load_happs_scores, df_sentiment
from data_mountain_query.counters import lang_dict
from data_mountain_query.connection import get_connection
from data_mountain_query.query import get_tweets
from data_mountain_query.parsers import load_ngrams_parser, parse_ngrams_tweet
import os

# -------------------------
# Load Dates from CSV
# -------------------------
input_csv = "/gpfs2/scratch/pwormser/research/shuffled_dates_2014.csv"  # Replace with actual path
output_csv = "30days_tweets.csv"

# Read dates from CSV into a list
date_df = pd.read_csv(input_csv, parse_dates=["Date"])  # Ensure 'date' column exists
dates = date_df["Date"].tolist()


lang = "en"
collection, client = get_connection(geotweets=True)

# Prepare storage for aggregated results
all_tweets = []

# Loop through each date
for current_date in dates:
    start_date = current_date
    end_date = current_date + timedelta(days=1)  # Next day

    print(f"Fetching tweets for {start_date.date()}...")

    query = {
        "tweet_created_at": {"$gte": start_date, "$lt": end_date},
        "fastText_lang": lang,
    }

    # Fetch tweets
    tweets = get_tweets(collection, query, limit=10)

    ngrams_parser = load_ngrams_parser()
    tweets = [{**tweet, "counters": parse_ngrams_tweet(tweet, ngrams_parser)} for tweet in tweets]

    # Convert to DataFrame
    df = pd.DataFrame(tweets)

    # Append to results
    all_tweets.append(df)


final_df = pd.concat(all_tweets, ignore_index=True)
print(final_df)
#final_df.to_csv(output_csv, index=False)

Connecting on dm-mongovm-001
Index(['id', 'geo', 'fastText_lang', 'fastText_conf', 'tweet_created_at',
       'pure_text', 'counters', 'rt_text'],
      dtype='object')
0       {'coordinates': [-34.83202457, -7.11762873]}
1             {'coordinates': [106.79176, -6.60567]}
2             {'coordinates': [104.65881, -3.20032]}
3       {'coordinates': [110.37898193, -7.78252287]}
4                {'coordinates': [120.397, 17.6222]}
                            ...                     
9995            {'coordinates': [-2.88424, 54.0633]}
9996     {'coordinates': [-118.2436849, 34.0522342]}
9997     {'coordinates': [28.84006558, 41.01839928]}
9998          {'coordinates': [112.74015, -7.27768]}
9999         {'coordinates': [-3.542993, 51.676323]}
Name: geo, Length: 10000, dtype: object
                      id                                           geo  \
0     726761360902737900  {'coordinates': [-34.83202457, -7.11762873]}   
1     726761361196257300        {'coordinates': [106.79176, 